# Test Vector Leakage Assessment

An example.

### 1. Generating examples

In this example, there are two keys $k_a$ and $k_b$. For each of these 2 keys, there are $1000$ power traces (samples)
available, each with a length of $10$ sample points.

In [491]:
samples = 1000
sample_pts = 10

The samples for these imaginary traces will be simplified into normal distributions with mean $\mu$.

A first-order "leakage" will be simulated by taking $\mu_a$ (for key $k_a$) slightly bigger than $\mu_b$.

In [492]:
mu_a = 42.2
mu_b = 42

In [493]:
import numpy as np

# Refer to section statistical properties of ASCAD in books/data/ascad_normality.
sigma = .2

traces_key_a = np.array([np.random.normal(size=samples, scale=sigma) + mu_a for _ in range(sample_pts)], dtype=int)
traces_key_b = np.array([np.random.normal(size=samples, scale=sigma) + mu_b for _ in range(sample_pts)], dtype=int)

### 2. Slicing the input

The first step in TVLA is to split these samples into four disjoint slices, each with a length of $250$.

The $n$-th slice for key $k_a$ will be denoted as $A_n$.

In [494]:
def make_slice(traces: np.array):
    select = np.array(range(traces.shape[1]))
    np.random.shuffle(select)

    return np.array([traces[:, s] for s in np.array_split(select, 4)])

A = make_slice(traces_key_a)
B = make_slice(traces_key_b)

In [495]:
A.shape

(4, 10, 250)

### 3. Preparing the data

In this example, the t-test is used when performing TVLA. The t-test takes the sample mean, sample variance and some
value for $p$. It tests the null hypothesis that 2 independent samples have identical average (expected) values, and
returns whether the null hypotheses should be accepted.

Therefore, $A$ and $B$ need to be transformed to fit the input parameters of the t-test.

In [496]:
A_mv = np.moveaxis(np.array((A.mean(axis=2), A.var(axis=2))), 0, -1)
B_mv = np.moveaxis(np.array((B.mean(axis=2), B.var(axis=2))), 0, -1)

A_mv.shape

(4, 10, 2)

### 4. Applying TVLA

In [497]:
from scipy.stats import stats

def make_t_test(n):
    def test(a, b):
        ma, sa = a
        mb, sb = b

        s = np.sqrt((sa + sb) / 2)
        t = (ma - mb) / (s * np.sqrt(2 / n))

        return np.abs(t).astype(np.float64)

    return test

num_observations = A.shape[2]
t_test = make_t_test(num_observations)

def test_spt(test, left, right):
    print([(l, r) for l, r in zip(left, right)])

    return np.array([test(l, r) for l, r in zip(left, right)])

def device_fails(left, right, t):
    return np.array((left < t, right < t), dtype=int).any()

def tvla(test, left, right, p=1, debug=False):
    """
    Applies some given statistical test against the given samples.

    @param test: statistical test function that takes two samples and returns test values.
    @param left: Four sample sets containing s samples representing a power trace of length t.
    @param right: Two sample sets containing s samples representing a power trace of length t.
    @param p: Decide whether a device fails using (100 * p)%-confidence interval.
    @param debug: Set to True if this function should throw Exceptions upon improperly structured input.
    @return: Whether the device fails on (A against A, A against B).
    """
    if debug:
        assert len(left) == 4
        assert len(right) == 2

    # Four different samples from distribution A.
    a, b, c, d = left
    # Two different samples from distribution B. x and y are compared to a and b, respectively.
    _, _, x, y = right

    # Test A against A.
    aa1 = test_spt(test, a, c)
    aa2 = test_spt(test, b, d)

    # Test A against B.
    ab1 = test_spt(test, a, x)
    ab2 = test_spt(test, b, y)

    t = np.percentile([aa1, aa2, ab1, ab2], p)

    # This value for t lets p% of the devices fail.
    return device_fails(aa1, aa2, t), device_fails(ab1, ab2, t)

tvla(test=t_test, left=A_mv, right=B_mv, p=5)

[(array([41.796   ,  0.162384]), array([41.816   ,  0.150144])), (array([41.848   ,  0.128896]), array([41.86  ,  0.1204])), (array([41.824   ,  0.145024]), array([41.832   ,  0.139776])), (array([41.808   ,  0.155136]), array([41.84  ,  0.1344])), (array([41.864   ,  0.117504]), array([41.86  ,  0.1204])), (array([41.868   ,  0.114576]), array([41.856   ,  0.123264])), (array([41.84  ,  0.1344]), array([41.844   ,  0.131664])), (array([41.848   ,  0.128896]), array([41.852   ,  0.126096])), (array([41.88  ,  0.1056]), array([41.836   ,  0.145104])), (array([41.82  ,  0.1476]), array([41.832   ,  0.139776]))]
[(array([41.836   ,  0.137104]), array([41.828   ,  0.142416])), (array([41.82  ,  0.1476]), array([41.86  ,  0.1204])), (array([41.824   ,  0.145024]), array([41.852   ,  0.126096])), (array([41.844   ,  0.131664]), array([41.844   ,  0.131664])), (array([41.868   ,  0.114576]), array([41.88  ,  0.1056])), (array([41.876   ,  0.108624]), array([41.84  ,  0.1344])), (array([41.888

(True, False)